In [1]:
import pandas as pd
import numpy as np
import re
from pprint import pprint
import collections
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers
import json
from nltk.corpus import stopwords

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import os

Using TensorFlow backend.
C:\Users\erajkovic\Anaconda3\envs\tensorflow_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\erajkovic\Anaconda3\envs\tensorflow_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\erajkovic\Anaconda3\envs\tensorflow_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users

In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['http', 'bully', 'bullying', 'rt', 'httplink', 'mkr'])
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
df = pd.read_csv('../text_twitter_type_raw.csv')
print(df.Type.unique())

#df = df.loc[df['Type'].isin(types)]
print(df)


['none' 'sexism' 'racism']
                                                    Text    Type     Label
0      @username @username @username I read them in c...    none  negative
1      @username Now you idiots claim that people who...    none  negative
2      RT @username Call me sexist, but when I go to ...  sexism  positive
3      @username Wrong, ISIS follows the example of M...  racism  positive
4                                 #mkr No No No No No No    none  negative
...                                                  ...     ...       ...
16846  Feeling so sorry for the girls, they should be...    none  negative
16847  #MKR 'pretty good dishes we're happy with' - O...    none  negative
16848  RT @username Deconstructed lemon tart!....can ...    none  negative
16849  @username @username @username You are too stup...    none  negative
16850  And before you protest that you're *not* mad, ...    none  negative

[16851 rows x 3 columns]


In [5]:
"""remove_rt = lambda x: re.sub('RT @\w+: ', "", x)
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",x)
df.Text = df.Text.map(remove_rt).map(rt)

print(df.Text)"""

# Convert to list
data = df.Text.values.tolist()

data = [re.sub('RT @\w+: ', "",sent) for sent in data]

#remove links
data = [re.sub(r"http\S+", "", sent) for sent in data]


# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['I read them in context.No change in meaning. The history of Islamic slavery. '
 'HTTPLINK']


In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=False))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['read', 'them', 'in', 'context', 'no', 'change', 'in', 'meaning', 'the', 'history', 'of', 'islamic', 'slavery', 'httplink']]


In [26]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

C:\Users\erajkovic\Anaconda3\envs\tensorflow_env\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['rt', 'call', 'me', 'sexist', 'but', 'when', 'go', 'to', 'an', 'auto', 'place', 'id', 'rather', 'talk', 'to', 'guy']


In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def remove_no_dict_words(texts):
    return [[word for word in doc if word in glove_dict] for doc in texts]
    
GLOVE_DIM=100
glove_file = 'C:/Users/erajkovic/Downloads/glove.twitter.27B/glove.twitter.27B.100d.txt'
glove_dict = []
NB_WORDS = 4525   # duljina file-a vocab
glove = open(glove_file, encoding="utf8")
for line in glove:
    values = line.split()
    word = values[0]
    glove_dict.append(word)
glove.close()
print(glove_dict[:3])
    

['<user>', '.', ':']


In [8]:
# Remove Stop Words
data_words_nodict = remove_no_dict_words(data_words)
data_words_nostops = remove_stopwords(data_words_nodict)

# Form Bigrams
#data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized = remove_stopwords(data_lemmatized)


print(data_lemmatized[:1])


[['read', 'context', 'change', 'mean', 'history', 'islamic', 'slavery']]


In [11]:
list_of_strings = []
for strings in data_lemmatized:
    concatenated_string = " ".join(strings)
    list_of_strings.append(concatenated_string)
    

with open('../cleaned_english_tweets_newData.txt', 'w', encoding='utf-8') as filehandle:
    for listitem in list_of_strings:
        if len(listitem)>0:
            print(listitem)
            filehandle.write('%s\n' % listitem)

read context change mean history islamic slavery
idiot claim people try stop become terrorist make terrorist brain dead
call sexist go auto place rather talk guy
wrong isis follow example mohamme quran exactly
mkr
saudi preacher rape torture year old daughter death release
nooo sexist woman bad driver
go make pancake hve strawberry hve banana mkr
dare feeling fantastic way dehumanize
wrong girl win wayne rooney street striker
run block manually
good muslim good bad religion
help ask pass judgment come look help
call care human life idiot genocidal daesh understand
otoh eventually wait harper randi harper still know adjust
roflmao putin delusion factory run full speed putin pay troll distribute
kat amp andre drown river amp chance save type sandwich
literally site today
juxtaposition
woo wait see happen mkr
tend document hair color experiment later gt gt pigtail pic year old
accord holly geordie shore sassy halfway slut classy mkr mkr
vomit
aww look forward ash camilla butting head prom

offence woman poor driver
embarrassed mkr
hate kat andre mkr
kat andre one trend mkr
gun draw ferguson pd
low give special insight pulpit privilege abuse free living premium
razer chroma
seem understand care less australian law
learn propaganda
islam lapidation amputation crucifixion terrorisme barbarie
vickys job stand hereby scare bear away mkr
mkr blonde keep pronounce porno model incorrectly actually harsh sorry lady porn biz
isis begin dig trench mosul fear aggressive assault iraqi force stronghold ruda
non israeli non jew stand know truth savagery israel amp jew
never stop benefit stop immoral
mmmm yum love chestnut gnocchi roasted creamy caramel aussie food
obviously shred evidence meat conspiracy theory lunatic
o_o appreciate greatly go release financial info soon
wait see winner mkr
catch mkr disgraceful behaviour kat andre strategic scoring save forever
love let sheri emilie go mkr
breakfast serve team head mkr
hope momentary trend able vote repub
joke bullshit score mkr
youn

listen talk harassment important going help
dude go culinary school
clear picture show repress impoverished area marching backwards time
kat andre partially critical know insta restaurant worth mkr
disappointed big fan blonde like much kat mkr
gater get excited talk reddit pretty funny sure go go tell friend totally rekt
free
call sexist yet woman roll good burrito chipotle year
never
yahya flip pkk communist clearly long true propaganda
outrage kat andre kick strategic mkr
sexist want watch ghostbuster film woman
awww vivek make new friend
put bag head kat want see glee people misfortune awful leave mkr
guy remember weev release illegally obtain info sex life full disclosure mailing list bcz uppity
bitter geno
medium talk head ever broach subject islam incompatible free speech democracy pc coward
ampe gurl mkr socialite
come bruh
open world stuff confuse much need guide path
help find friend sister signal boost shit get home
look forward kat andre boot soon bring back together mkr
bas

sexist fall asleep female drive xan
meh myer regain form forget ever exist armia lemieux great asset
mkr many round watch get real competition
well buy case sooo
go social club see woman main bar mad right
follow ايلي ايليا twi insta mkr
sexist girl talk football put mac product buy stay lane
jihadi group want reinstate slavery present quran approve mohamme practice
word say islam vile
grammer islam phobia medical condition truth
compare woman man profession experience sexist
computer programming sexist fuck womanly job
question german criminal prosecutor announce investigation islam incitement hatred islam
mkr year seriously watch instant restaurant round mkr
agree idea compliment absurd important compliment
feel great twitter much lately catch later
hate much sexist fuck lady awful
mosul isis terrorist corpse hospital corpse pick commander execute
head gdc hour japanese kit kat purse mug bro
sexist woman open door pay bill drive still beleive chivalry
well accurate ill dropping tomor

go ignore reality keep spout left wing dogma engage favorite flavor hate
limitation certain regulation flirt talk douchbag
egg cook mkr
new name btw punch inane tool use silence dissent fact logic
mkr crap dish good one right kitchen rule silly
show rig mkr ai good view
eva debra seem lovely mkr
success
legitimate complaint even hackernew say work fine
real contribution muslim world islam
call sexist label old fashion sad sight world drunk woman sadden hell outta
look towel wrap head pretty freaky
possible super low retention graph guess wsp file delete update minute
kurd root isis western
creator website argue make rape legal
get weird insult ever see sling mra gater
quran hadith
hate know thank rub hypocrite face shortly
amused popular tweet unexpected
dad
think muslim woman know prophet call stupid
time go rob hat mkr
tonight mkr result prove smug arsehole prosper mkr
worry back publish everywhere try discredit
man really good spouting sexism second guy sexist think man woman good d

...

nusra behead man savage way isis idlib syria rip
like dry sausage mkr
google false rape accusation work
ball put bloody kat still mkr
rest world cater go enjoy shoot huge mouth woman first foremost
arabiya russia send arm yeman port air missile
funny thing
see day ago brilliant need
african slave slave muslim
realize powerful feminism really
care none business religion want
south african happy go soon mkr
omg shut bianca mkr
ugh year
call sexist man telecast woman sport woman telecast man sport sorrynotsorry
tweet cap pub thing
recently buy kingpin card always buy nvidia
fact pedophile prophet clearly state intention hadith
read statistic look percentage world muslim want kill
nice try right speak move fucking mar always gon say get say
miss person dolore dennis lbs last see today blk baker st info pls call
christian nation say want christian amp jesus muslim kill blasphemy
mother tell mom leave money counter also ask sandwich
sexist woman girl never rap
woman write woman tech book sto

bowl cornflake wheat perfect come mkr
unbelievable way journalist fall make excuse barbarity islam read quran
western medium put head stake
miss guy mkr mkr
totally agree hope soon mkr
mkr channel stick channel contestant play tennis probably cahoot lift rating
dead shoot satirical publication office paris prove
go nuts mention wtf go away
several strand pearl love
suddenly muslim neighbor persecute steal property
craving meet mkr masterchef foodporn foodie
say islam way life mean human freedom forbid inhuman law criminal follow
top kek
go home kat cunt face bitch mkr
mosul artillery shell isis terrorist position today kurdistan
nooooo miss critique entree miss main dessert mkr
add filter toggle show phone verify acct client require update natch
howl pain ball get kick sexist girl guy physically
use killer blonde hash tag mkr
fuck hard rusty chainsaw round instant restaurant mkr
lol people give platform ignore detractor
side note holy shit know many follower
go die get isis well shia m

listen right glorious
kat dealer obviously deliver mkr
dumb asshole legalize slavery rape slave girl pedophile call great
thing woman give fuck say body
case twitter probably current
gray sweat pant sweatshirt
bad take call sexist want ill take woman cook day
boom officially turn wake mkr mkr
believe say people read email people see picture naked body
rise josh favourite team mkr
comcast time warner merger still bad deal consumer report policy
flyer never get phase sync great forwards great never find
love tweet sassy girl cooking mkr tonight
let hope goat happy mkr
verse prove sexism islam
sexist fully believe support idea gender equality woman really drive
remove fucking hat table mkr
reality tv disgust sexist reality young girl role model
ask find employer value properly
tonight dessert call remain day mkr
quran writer imagine god egotistical wanting worship bow unsophisticated
whittle short month mkr
lol colin mkr
arabian peninsula belong muslim begin exterminate give
mkr promo gir

sassy gurl ai sassy bitchy mkr
agree hadith classify sound hadith writer classify weak
isis practice genocide prophet mohamme practice
let clear existence address problem affect
quran full nonsense amp prove rest case
yw know man woman different job pay
bugger forget simon reeve sbs tonight mkr
dictionary
way harsh bank account application
look slum people force go mosque gun point daesh create hundred thousand atheist
interesting vedas gita say beat wife ghandi prophet hinduism
guy comedy gold encouraging wtf
break eliminate mkr wait
get fuck shit fuck sexist assume male misogynist
rose red hair blue follow jerk else ill block happy valentine day
shame katie nikki kat go mkr
winston churchill say islam
also fellow sj butthurt generally frown descriptor
take tiny village loose major city
hate girl bad mood try say hate girl
look dude black woman shitty world white supremacist hierarchy maybe try solidarity
deconstruct lemon tart mkr think drop floor scrape bowl
difficult sometimes alwa

fate feminist
stupid sexist sister hell spoil daddys lil girl work struggle day
leo stoic guard night
come moron let go verse quran start afraid
fkn die kat andre mkr
sexist woman jacket look weird
sexist help cringe woman talk football
team serve cold pizza breakfast item mkr
mkr vida
love save sheri amp emilie mkr
agree sure page
see fact rofl emotion get way far often sexist state see fact
go intercept dish know say
call kat andre despise people australia bit stretch person clearly still tony abbot mkr
somehow blonde triumph bugger mkr
hope people remember turkish president say isis terrorist angry people gather together
really want hot cake mkr
sexist bitch drive shit
suggestion let make manu colin main judge show mkr
muslim abuse murder minority religion continue muslim majority country islam pakistan syria
accept evidence bother ask
go em amp shez mkr sudden death beckon
lynn tony need well microwave skill stay competition use packet food well mkr
political correctness fertilizer

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
tfidf = TfidfModel(corpus)
corpus = tfidf[corpus]

# View
print(corpus[:1])
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus]

[[('change', 0.37060606477621677),
  ('context', 0.48977123627178026),
  ('history', 0.3922125561810948),
  ('islamic', 0.3601054344421957),
  ('mean', 0.3043658475752585),
  ('read', 0.311326094396853),
  ('slavery', 0.38691880480067636)],
 [('become', 0.32510915983997274),
  ('brain', 0.35300551855135176),
  ('claim', 0.29651353920755),
  ('dead', 0.32991926670922195),
  ('idiot', 0.2881142335127067),
  ('make', 0.18707037140597874),
  ('people', 0.18698438594972708),
  ('stop', 0.24803937390593642),
  ('terrorist', 0.5492947774085544),
  ('try', 0.23813348585521557)],
 [('auto', 0.5642247228075395),
  ('call', 0.2715005986084263),
  ('go', 0.21741389519734242),
  ('guy', 0.3302962219843349),
  ('place', 0.3908610815933511),
  ('rather', 0.4086416260492427),
  ('sexist', 0.2033203968630194),
  ('talk', 0.3007888688448862)],
 [('exactly', 0.42612413202277427),
  ('example', 0.44346905637810835),
  ('follow', 0.38317084765759685),
  ('isis', 0.286696914743161),
  ('mohamme', 0.35098283

In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=100,
                                           alpha='auto',
                                           per_word_topics=True)

In [11]:
# Print the Keyword in the n topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.012*"sexist" + 0.010*"get" + 0.009*"say" + 0.008*"girl" + 0.008*"see" + '
  '0.008*"make" + 0.007*"know" + 0.006*"hate" + 0.006*"good" + 0.006*"time"'),
 (1,
  '0.013*"kat" + 0.012*"woman" + 0.010*"islam" + 0.009*"muslim" + '
  '0.008*"people" + 0.008*"amp" + 0.007*"andre" + 0.007*"want" + 0.006*"man" + '
  '0.006*"well"'),
 (2,
  '0.017*"go" + 0.014*"think" + 0.009*"look" + 0.008*"call" + 0.007*"still" + '
  '0.007*"talk" + 0.006*"way" + 0.006*"feminist" + 0.006*"ever" + '
  '0.005*"team"')]


In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.920877223328315

Coherence Score:  0.23499939211875778


In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.180327 -0.081622       1        1  39.648776
0     -0.171474 -0.094953       2        1  37.662672
2     -0.008854  0.176575       3        1  22.688552, topic_info=           Term        Freq       Total Category  logprob  loglift
19           go  143.000000  143.000000  Default  30.0000  30.0000
147       think  125.000000  125.000000  Default  29.0000  29.0000
99          kat  198.000000  198.000000  Default  28.0000  28.0000
23       sexist  169.000000  169.000000  Default  27.0000  27.0000
44        woman  175.000000  175.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
68         help   33.785512   34.570830   Topic3  -5.5445   1.4603
352       score   30.345564   31.057186   Topic3  -5.6519   1.4601
372        home   32.112747   32.875203   Topic3  -5.5953   1.4598
659        name   31.409511   32.171989   Topic3  -5.6174   1.4593
1265  breakfast   30.725711   31.528062   Topic3  -5.6394   1.4575

[123 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1898      1  0.991144   also
95        1  0.997162    amp
96        1  0.992952  andre
504       1  0.976330  annie
410       3  0.978774   back
...     ...       ...    ...
53        3  0.995211    way
603       1  0.998258   well
44        1  0.993545  woman
586       2  0.980236   work
40        2  0.983865   year

[93 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

In [14]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
os.environ['MALLET_HOME'] = 'C:/Users/erajkovic/Downloads/mallet-2.0.8/mallet-2.0.8'
mallet_path = "C:/Users/erajkovic/Downloads/mallet-2.0.8/mallet-2.0.8/bin/mallet" # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=3, id2word=id2word)

C:\Users\erajkovic\Anaconda3\envs\tensorflow_env\lib\site-packages\smart_open\smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [15]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  [('lol', 0.037037037037037035),
   ('view', 0.031746031746031744),
   ('islam', 0.031746031746031744),
   ('good', 0.026455026455026454),
   ('relevant', 0.026455026455026454),
   ('bullshit', 0.021164021164021163),
   ('rig', 0.021164021164021163),
   ('work', 0.021164021164021163),
   ('answer', 0.015873015873015872),
   ('happen', 0.015873015873015872)]),
 (1,
  [('love', 0.05069124423963134),
   ('stuff', 0.02304147465437788),
   ('lie', 0.02304147465437788),
   ('roflmao', 0.018433179723502304),
   ('block', 0.018433179723502304),
   ('omg', 0.018433179723502304),
   ('true', 0.018433179723502304),
   ('wat', 0.013824884792626729),
   ('kat', 0.013824884792626729),
   ('weird', 0.013824884792626729)]),
 (2,
  [('sexist', 0.039603960396039604),
   ('fuck', 0.034653465346534656),
   ('shit', 0.024752475247524754),
   ('rofl', 0.024752475247524754),
   ('awesome', 0.024752475247524754),
   ('liar', 0.019801980198019802),
   ('gt', 0.019801980198019802),
   ('link', 0.019801980

In [16]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:

model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=24, step=1)

C:\Users\erajkovic\Anaconda3\envs\tensorflow_env\lib\site-packages\smart_open\smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# Show graph
limit=24; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topic =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[0]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)


In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
print(topic_counts)

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics